In [ ]:
# PFM VEHICLE COLLISION

## Desde 2011, tras la aprobación de la ley local Nº11, la ciudad de New York publica información relacionada con las colisiones producidas en la ciudad. Esta información se ejecuta manualmente todos los meses y la Unidad TrafficStat revisa la información antes de publicarla en el sitio web del NYPD. 

## En el dataset contiene más de 400k registros. Cada registro representa una colisión en NYC con información de dia y fecha, barrio, cruce de calles, número de heridos, tipo de vehiculos involucrados y los motivos de la colisión.

## Cada año hay más de 170.000 accidentes en la ciudad de NYC en sus cinco distritos: Manhattan, Brooklyn, Staten Island, Bronx y Queens. 
## Los objetivos del análisis de este dataset son: 
   ## - 	Identificar las causas por las que se produce un accidente
   ## - 	Identificar los cruces y zonas más peligrosas
   ## - 	Identificar las zonas más peligrosas o con más colisiones
   ## -	Clasificar las localizaciones a través del algoritmo HDBSCAN


In [ ]:
# LIMPIEZA DEL DATASET

## Importación de las bibliotecas necesarias
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import numpy as np
import calendar
from datetime import datetime
import csv
import reverse_geocoder as rg
import ast

In [ ]:
## Cargar del fichero con pandas
data=pd.read_csv('D:/Mireia D/database.csv',sep=',',parse_dates=['DATE','TIME'],low_memory=False)

In [ ]:
## Hay 140.000 observaciones sin información de distrito [BOROUGH]. Utilizando algoritmo geocoder reverse, he conseguido completar el 92% de esta información en el dataset, ya que si tenemos las coordenadas de cada observación.
## Elimino también todas las observaciones sin localización ya que el objetivo del análisis es encontrar las localizaciones más seguras y peligrosas. 
data.isnull().sum()

In [ ]:
#### Eliminar columnas sin localización
data=data.dropna(subset=['LOCATION'])

In [ ]:
#### Crear otro datafame solo con información de localiazción y distrito, y elimino los que sí tienen informado el distrito.
df_geo=data[['LOCATION','BOROUGH']].copy()
df_geo=df_geo[pd.isnull(data['BOROUGH'])]

In [ ]:
#### Lista de coordenadas para que el algoritmo busque la información del distrito
coordenadas = df_geo['LOCATION'].values
coordenadas = list(ast.literal_eval(','.join(i.strip() for i in coordenadas)))

In [ ]:
#### Aplicar la búsqueda de dirección para las coordenadas de nuestro dataset sin info de distrito
results = rg.search(coordenadas)

In [ ]:
#### Crear fichero para guardar la información del distrito y sus coordenadas
output_filename = 'D:/Mireia D/Distritos.csv'

In [ ]:
#### Rellenar fichero Distritos con la informacion de cada distrito dada unas coordenadas
rows = []
for idx, city in enumerate(coordenadas):
    write_row = []
    lat=city[0]
    lon=city[1]
    gdata = results[idx]
    rows.append([lat,lon,gdata['admin2']])
csvwriter = csv.writer(open(output_filename,'w+'),delimiter=' ')
csvwriter.writerows(rows)

In [ ]:
#### Cargar el fichero con la info de distrito y la unimos al resto de información por accidente
distrito=pd.read_csv('D:/Mireia D/Distritos.csv',sep=' ',header=None,low_memory=False)
distrito=distrito.rename(columns={0:"LATITUDE",1:"LONGITUDE",2:"BOROUGH",3:"COORDENADAS"})
distrito['COORDENADAS']=[', '.join(str(x) for x in y) for y in map(tuple, distrito[['LATITUDE', 'LONGITUDE']].values)]
distrito=distrito.drop_duplicates(subset=['COORDENADAS'])

In [ ]:
#### Eliminar cuando el distrito que devuelve es Hunterdon county ya que sus coordenadas estan fuera de la ciudad de NY
#### Sustituir los barrios que devuelve el algoritmo por los que utiliza el fichero original

distrito=distrito[distrito.BOROUGH != 'Hunterdon County']

distrito=distrito.replace(["Queens County","Nassau County"],'QUEENS')
distrito=distrito.replace(["Kings County",],'BROOKLYN')
distrito=distrito.replace(["New York County","Bergen County"],'MANHATTAN')
distrito=distrito.replace(["Bronx","Westchester County"],'BRONX')
distrito=distrito.replace(["Richmond County","Hudson County","Middlesex County"],'STATEN ISLAND')
distrito[:5]

In [ ]:
#### Combinar ambos ficheros
result = pd.merge(data, distrito[['LATITUDE','LONGITUDE','BOROUGH']], on=['LATITUDE','LONGITUDE'],how='left')
result[:5]

In [ ]:
#### Completar BOROUGH_x con BOROUGH:_y
result.loc[result['BOROUGH_x'].isnull(),'BOROUGH_x']=result.BOROUGH_y
result[:5]

In [ ]:
####Eliminar variable BOROUGH_y del dataframe y renombramos las columnas del dataset
#También eliminamos Person injured y person killed porque ya tenemos otras tres columnas que dan esa misma información de manera desagregada.
result=result.drop(result.columns[[11,12,29]],axis=1)
result.columns=['UNIQUE_KEY','DATE','TIME','BOROUGH','CODE','LATITUDE','LONGITUDE','LOCATION','STREET','CROSS_STREET','OFF_STREET','PEDESTRIAN_INJURED','PEDESTRIAN_KILLED','CYCLIST_INJURED','CYCLIST_KILLED','MOTORIST_INJURED','MOTORIST_KILLED','TYPE_1','TYPE_2','TYPE_3','TYPE_4','TYPE_5','FACTOR_1','FACTOR_2','FACTOR_3','FACTOR_4','FACTOR_5']
#Filtrar coordenadas 0,0
result=result[result.LATITUDE != 0.0] 
result=result[result.LONGITUDE != 0.0]


In [ ]:
#### Había 47.602 observaciones sin información de ZIP COCE o BOROUGH. Utilizando algoritmo geocoder reverse, he conseguido completar el 92% de esta información en el dataset
result.isnull().sum()

In [ ]:
#### Para analizar si existen dias de la semana, horas del dia o meses más peligrosos, voy a crear estas varaibles en el dataset para facilitar este análissi posterior
df=pd.DataFrame(result)

df['YEAR']=df['DATE'].dt.year
df['MONTH']=df['DATE'].dt.month
df['NAMEMONTH']=df['MONTH'].apply(lambda x: calendar.month_abbr[x])
df['NAMEWEEK']=df['DATE'].dt.weekday_name
df['YEARMONTH'] = df['DATE'].apply(lambda x:x.strftime('%Y-%m'))
df['HOUR']=df['TIME'].dt.hour

In [ ]:
## Se crean variables resumen de heridos y fallecidos por tipo de persona involucrada, con la intención de poder analizar mejor esta información 
#### Crear variables que sumen el total de personas heridas y fallecidas segun tipo de persona involucrada en el accidente
df['PEDESTRIAN']=df['PEDESTRIAN_INJURED'] + df['PEDESTRIAN_KILLED']
df['CYCLIST']=df['CYCLIST_INJURED'] + df['CYCLIST_KILLED']
df['MOTORIST']=df['MOTORIST_INJURED'] + df['MOTORIST_KILLED']
#### Crear variable que sume el total de personas heridas y fallecidas independientemente de si es peaton, conductor o ciclista
df['PERSON']=df['PEDESTRIAN'] + df['CYCLIST'] + df['MOTORIST']

In [ ]:
## Se crea variable que nos indique el número de vehiculos involucrados en cada accidente 
#### Crear variable que nos indique el número de vehiculos involucrados
type_vehicle=[x for x in df.columns if 'TYPE' in x]
df['N_VEHICLE']=df[type_vehicle].apply(lambda x: x.count(),axis=1)

In [ ]:
## Se crea variable que nos inqique la gravedad del accidente
#### clasificando cada accidente en sin heridos, con heridos o con personas fallecidas
heridos=[x for x in df.columns if 'INJURED' in x]
muertes=[x for x in df.columns if 'KILLED' in x]
df['GRAVEDAD']='Sin_heridos'
df.loc[df[muertes].sum(axis=1)>0,'GRAVEDAD']='Muertes'
df.loc[(df[muertes].sum(axis=1)==0)&(df[heridos].sum(axis=1)>0),'GRAVEDAD']='Heridos'
df.GRAVEDAD.value_counts()

In [ ]:
## Normalización de los tipos de vehiculos
#### Renombrar algunos tipo de vehiculo mal escritos y cambiar a mayusculas
for columns in df[type_vehicle]:
    df[type_vehicle]=df[type_vehicle].apply(lambda col:col.str.upper())
    df[type_vehicle]=df[type_vehicle].replace(["ABULA","AM","AMB","AMBUL","AR"],'AMBULANCE')
    df[type_vehicle]=df[type_vehicle].replace(["BU","BSD"],'BUS')
    df[type_vehicle]=df[type_vehicle].replace(["FB","FDNY","FIRET","FIRE"],'FIRE TRUCK')
    df[type_vehicle]=df[type_vehicle].replace(["SCOOTER","MOTOR"],'MOTORCYCLE')

In [ ]:
## Normalización de las calles
STREET_GROUP=[x for x in df.columns if 'STREET' in x]
for columns in df[STREET_GROUP]:
    df[STREET_GROUP]=df[STREET_GROUP].apply(lambda col:col.str.upper())

normalization_street={'AVENUE':'AV',
    'AVE':'AV',
    'AVNUE': 'AV',
    'STREET': 'ST',
    'ROAD': 'RD',
    'BOULEVARD': 'BLVD',
    'PLACE': 'PL',
    'PLAZA': 'PL',
    'SQUARE': 'SQ',
    'DRIVE': 'DR',
    'LANE': 'LN',
    'PARKWAY': 'PKWY',
    'TURNPIKE': 'TP',
    'TERRACE': 'TER',
    '1ST': '1',
    '2ND':'2',
    '3RD': '3',
    '1TH': '1',
    '2TH': '2',
    '3TH': '3',
    '4TH': '4',
    '5TH': '5',
    '6TH': '6',
    '7TH': '7', 
    '8TH': '8',
    '9TH': '9',
    '0TH': '0',              
    'WEST ': 'W ',
    'NORTH ': 'N ',
    'EAST ': 'E ',
    'SOUTH ': 'S '}

#### Realizar reemplazo con los valores de normalización
for column in STREET_GROUP:
    df[STREET_GROUP]=df[STREET_GROUP].replace(normalization_street,regex=True)

In [ ]:
# ANÁLISIS EXPLORATORIO 
## ¿Cuales son los tipos de vehiculo más involucrados en accidentes?

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
df_type = pd.DataFrame([df.loc[:,x].value_counts().nlargest(10) for x in type_vehicle]).fillna(0).sum().sort_values(ascending=True)
df_type.plot(kind='barh',ax=ax)
ax.set_title('Tipos de vehiculos en accidentes NYC')
ax.set_xlabel('Nº accidentes')
plt.show()
